In [1]:
import requests

import numpy as np
import pandas as pd

from pycoingecko import CoinGeckoAPI

from bs4 import BeautifulSoup as bts

### Why isn't this a script?

We are only generating the dataset once. We don't need to write a script that does that is properly formatted and tested, we can do it using a notebooks and test our results at the end.

### Using the Coin Gecko API